In [ ]:
#TODO: Think of a test-driven development approach to evaluate the model.
#TODO: Add evaluation metrics/tests with promptimize/RAGAS and other methods
#TODO: Add TruLens evaluation with RAG triad & evaluation functions. (e.g. CoT): Feedback functions. 
# See: https://learn.deeplearning.ai/building-evaluating-advanced-rag/lesson/3/rag-triad-of-metrics

In [18]:
!pip install azure-identity azure-keyvault-secrets

In [28]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

key_vault_name = "kv-bsauwmno"
kv_uri = f"https://{key_vault_name}.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

# Now you can use neo4j_url, neo4j_port, and neo4j_password in your application
neo4j_url = client.get_secret("NEO4JURL").value
neo4j_user = client.get_secret("NEO4JUSER").value
neo4j_password = client.get_secret("NEO4JPASSWORD").value
openai_key = client.get_secret("OPENAIKEY").value

In [47]:
import pandas as pd 
from langchain.embeddings import OpenAIEmbeddings

embedding_model = "text-embedding-ada-002"

openai = OpenAIEmbeddings(
    openai_api_key=openai_key,
    model=embedding_model
)


def cast_col(df: pd.DataFrame, col: str, cast_type: str) -> pd.DataFrame:
    df[col] = df[col].astype(cast_type)
    return df

def embed_column(
        df: pd.DataFrame, 
        col_name: str, 
        col_name_embedded: str) -> pd.DataFrame:
    docs = list(df[col_name])
    embeddings: List[List[float]] = openai.embed_documents(texts=docs)
    df[col_name_embedded] = embeddings
    return df 

question_df = (
    pd.read_excel(
        './data/qa_dataset.xlsx', 
        engine="openpyxl",
        sheet_name=1
    )
    .pipe(pd.DataFrame.dropna, subset=['question_id', 'question', 'answer_id', 'group'])
    .pipe(cast_col, col='answer_id', cast_type='int64')
    .pipe(cast_col, col='question_id', cast_type='int64')

)

answer_df = (
    pd.read_excel(
        './data/qa_dataset.xlsx', 
        engine="openpyxl",
        sheet_name=2
    )
    .pipe(pd.DataFrame.rename, columns={
        "question": "archetype_question",
        "answer": "archetype_answer"
    })
    .pipe(
        embed_column,
        col_name="archetype_question", 
        col_name_embedded="archetype_question_embedding"
    )
)

qa_df = question_df.merge(answer_df, on="answer_id", how="left")

In [48]:
qa_df

,question_id,group,question,answer_id,archetype_question,archetype_answer,source,archetype_question_embedding
0,1,time,Is het mogelijk om 8 uur na de geboorte met mi...,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...,"[0.01071266053997743, -0.022445575617780624, -..."
1,2,time,Kan ik met mijn pasgeboren baby 12 uur na de g...,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...,"[0.01071266053997743, -0.022445575617780624, -..."
2,3,time,Is het toegestaan om 10 uur na de bevalling he...,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...,"[0.01071266053997743, -0.022445575617780624, -..."
3,4,time,Mag ik 15 uur na de geboorte van mijn kind het...,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...,"[0.01071266053997743, -0.022445575617780624, -..."
4,5,time,Kan ik 30 uur na de geboorte met mijn baby het...,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...,"[0.01071266053997743, -0.022445575617780624, -..."
...,...,...,...,...,...,...,...,...
594,595,very_low_literacy,Hoe kan ik mijn baby gezond houden?,4,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,https://www.azstlucas.be/onderzoek-en-behandel...,"[0.03156107295980677, -0.019481835914004948, 0..."
595,596,very_low_literacy,Wat moet ik doen als mijn baby ziek wordt?,4,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,https://www.azstlucas.be/onderzoek-en-behandel...,"[0.03156107295980677, -0.019481835914004948, 0..."
596,597,very_low_literacy,Hoe kan ik ervoor zorgen dat mijn baby goed gr...,4,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,https://www.azstlucas.be/onderzoek-en-behandel...,"[0.03156107295980677, -0.019481835914004948, 0..."
597,598,very_low_literacy,Wat als ik denk dat er iets mis is met mijn baby?,4,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,https://www.azstlucas.be/onderzoek-en-behandel...,"[0.03156107295980677, -0.019481835914004948, 0..."


In [36]:
# For every row in QA df
# - match the question_id 
# - for each archetype question, get the distance from the question to that archetype question 


In [41]:
from typing import List 
from neo4j import GraphDatabase, RoutingControl
from langchain.embeddings import OpenAIEmbeddings

embedding_model = "text-embedding-ada-002"

openai = OpenAIEmbeddings(
    openai_api_key=openai_key,
    model=embedding_model
)

def to_pandas(records: List[object]) -> pd.DataFrame:
    return pd.DataFrame.from_records([dict(r) for r in records])

def get_qa_db(driver, db='neo4j'): 
    records, _, _ = driver.execute_query(
        """
            MATCH (q:Question)--(a:Answer)
            RETURN
                q.question_id as question_id,
                q.text as question_text, 
                q.text_embedding as question_embedding, 
                a.answer_id as answer_id, 
                a.text as answer_text
        """,
        database_= db, 
        routing_= RoutingControl.READ,
    )
    return to_pandas(records)    

# Drop index using: DROP INDEX `vi_question_text_embedding_cosine`
with GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password)) as driver:
    qa_db_df = get_qa_db(driver)


qa_db_df

,question_id,question_text,question_embedding,answer_id,answer_text
0,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu..."
1,150,"Baby oké, nu naar huis?","[-0.0008756839387770835, -0.03697606429392978,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu..."
2,149,"Baby klaar, weg nu?","[-0.004362260274536889, -0.034600230348239086,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu..."
3,148,"Baby er, snel huis?","[-0.010511535975893377, -0.03074530274562924, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu..."
4,147,"Baby geboren, ik naar huis?","[0.002126995245116866, -0.0329753384590574, -0...",1,"Neen, een pasgeboren baby moet de eerste 24 uu..."
...,...,...,...,...,...
594,494,Is er iets dat ouders moeten doen of weten met...,"[0.013448605935884416, -0.013833217935967097, ...",4,Na de geboorte kunnen er problemen zoals aange...
595,495,Wat zijn de belangrijkste overwegingen bij het...,"[0.022697007552032526, -0.012322678376457655, ...",4,Na de geboorte kunnen er problemen zoals aange...
596,496,Kunnen dezelfde zorgmaatregelen worden genomen...,"[0.013839255036708156, -0.021893038876721768, ...",4,Na de geboorte kunnen er problemen zoals aange...
597,497,Wat moeten ouders doen als ze zich zorgen make...,"[0.01546604698769678, -0.01740405202435655, 0....",4,Na de geboorte kunnen er problemen zoals aange...


In [67]:
qa_db_df.merge(
qa_df[[
    'answer_id', 
    'archetype_question', 
    'archetype_answer', 
    'archetype_question_embedding', 
    'group',
    'source'
]],
on="answer_id"
)

,question_id,question_text,question_embedding,answer_id,answer_text,archetype_question,archetype_answer,archetype_question_embedding,group,source
0,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",time,https://www.azstlucas.be/onderzoek-en-behandel...
1,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",time,https://www.azstlucas.be/onderzoek-en-behandel...
2,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",time,https://www.azstlucas.be/onderzoek-en-behandel...
3,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",time,https://www.azstlucas.be/onderzoek-en-behandel...
4,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",time,https://www.azstlucas.be/onderzoek-en-behandel...
...,...,...,...,...,...,...,...,...,...,...
89696,498,Is er een verschil in de beschikbaarheid van s...,"[0.0076592760011275065, -0.014072727384555625,...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",very_low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...
89697,498,Is er een verschil in de beschikbaarheid van s...,"[0.0076592760011275065, -0.014072727384555625,...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",very_low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...
89698,498,Is er een verschil in de beschikbaarheid van s...,"[0.0076592760011275065, -0.014072727384555625,...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",very_low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...
89699,498,Is er een verschil in de beschikbaarheid van s...,"[0.0076592760011275065, -0.014072727384555625,...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",very_low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...


In [60]:
answer_df[['archetype_question_embedding']]

,archetype_question_embedding
0,"[0.01071266053997743, -0.022445575617780624, -..."
1,"[0.023700067677154102, -0.01009931876876786, 0..."
2,"[0.014633797688849612, -0.024035821689667867, ..."
3,"[0.03156107295980677, -0.019481835914004948, 0..."


In [80]:
import numpy as np
from scipy.spatial.distance import cosine

def cosine_similarity_list(df: pd.DataFrame, col_name: str, col_a: str, col_b: str) -> pd.DataFrame:
    # Convert the lists in the columns to numpy arrays
    df[col_a + "_np"] = df[col_a].apply(lambda x: np.array(x))
    df[col_b + "_np"] = df[col_b].apply(lambda x: np.array(x))

    # Calculate the cosine similarity for each row and store in the new column
    df[col_name] = df.apply(lambda row: cosine(row[col_a + "_np"], row[col_b + "_np"]), axis=1)

    # Drop the temporary numpy array columns if not needed
    df.drop(columns=[col_a + "_np", col_b + "_np"], inplace=True)

    return df


# def pairwise_cosine(df: pd.DataFrame, col: str): 
#     arch_distances = []
#     for arch_emb in answer_df[['archetype_question_embedding']]: 
#         pairwise_distances = []


eval_df = (
        qa_db_df.merge(
        qa_df[[
            'answer_id', 
            'question_id', 
            'archetype_question', 
            'archetype_answer', 
            'archetype_question_embedding', 
            'group',
            'source'
        ]],
        how="left",
        on=["question_id", "answer_id"]
    )
    .pipe(
        cosine_similarity_list, 
        col_name="cosine_distance", 
        col_a="question_embedding",
        col_b="archetype_question_embedding"
    )

)

In [82]:
eval_df

,question_id,question_text,question_embedding,answer_id,answer_text,archetype_question,archetype_answer,archetype_question_embedding,group,source,cosine_distance
0,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.065959
1,150,"Baby oké, nu naar huis?","[-0.0008756839387770835, -0.03697606429392978,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.146958
2,149,"Baby klaar, weg nu?","[-0.004362260274536889, -0.034600230348239086,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.171928
3,148,"Baby er, snel huis?","[-0.010511535975893377, -0.03074530274562924, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.138975
4,147,"Baby geboren, ik naar huis?","[0.002126995245116866, -0.0329753384590574, -0...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.095512
...,...,...,...,...,...,...,...,...,...,...,...
594,494,Is er iets dat ouders moeten doen of weten met...,"[0.013448605935884416, -0.013833217935967097, ...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.111524
595,495,Wat zijn de belangrijkste overwegingen bij het...,"[0.022697007552032526, -0.012322678376457655, ...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.119217
596,496,Kunnen dezelfde zorgmaatregelen worden genomen...,"[0.013839255036708156, -0.021893038876721768, ...",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.124013
597,497,Wat moeten ouders doen als ze zich zorgen make...,"[0.01546604698769678, -0.01740405202435655, 0....",4,Na de geboorte kunnen er problemen zoals aange...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,"[0.03156107295980677, -0.019481835914004948, 0...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.127660


In [84]:
!pip install plotly

In [85]:
eval_df.head(10)

,question_id,question_text,question_embedding,answer_id,answer_text,archetype_question,archetype_answer,archetype_question_embedding,group,source,cosine_distance
0,50,Is het toegestaan om in een academisch ziekenh...,"[0.004562567368534598, -0.02192822616530894, -...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",location,https://www.azstlucas.be/onderzoek-en-behandel...,0.065959
1,150,"Baby oké, nu naar huis?","[-0.0008756839387770835, -0.03697606429392978,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.146958
2,149,"Baby klaar, weg nu?","[-0.004362260274536889, -0.034600230348239086,...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.171928
3,148,"Baby er, snel huis?","[-0.010511535975893377, -0.03074530274562924, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.138975
4,147,"Baby geboren, ik naar huis?","[0.002126995245116866, -0.0329753384590574, -0...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.095512
5,146,"Baby er, ik huis?","[-0.004205518623489822, -0.04441825906751255, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.133410
6,145,"Na baby, huis gaan?","[0.0029168254871937273, -0.03757585235274886, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.130254
7,144,"Baby gezond, huis gaan?","[0.012589036608492678, -0.023663884369102113, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.125775
8,143,"Baby en ik, weg nu?","[-0.014475592121221592, -0.03457990219259041, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.173437
9,142,"Baby klaar, naar huis?","[0.0018282463657134105, -0.03494941003810354, ...",1,"Neen, een pasgeboren baby moet de eerste 24 uu...",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...","[0.01071266053997743, -0.022445575617780624, -...",low_literacy,https://www.azstlucas.be/onderzoek-en-behandel...,0.134104


In [124]:
def calc_pairwise_distances(df: pd.DataFrame) -> pd.DataFrame: 
    # add constant distances. 
    for i, arch_qemb in enumerate(answer_df['archetype_question_embedding']):
        df[f'archq{i+1}_emb'] = df.apply(lambda _: arch_qemb.copy(), axis=1)
        df = cosine_similarity_list(
            df, 
            col_name=f'archq{i+1}_cosim',
            col_a='question_embedding',
            col_b=f'archq{i+1}_emb'
        )
    return df 
    
def get_shortest_dist(df: pd.DataFrame) -> pd.DataFrame: 
    min_question_ids = []
    min_cosims = []
    cosim_cols = [f'archq{a}_cosim' for a in answer_df['answer_id']]

    for i, obj in df.iterrows(): 
        cosim_vals = [obj[k] for k in cosim_cols]
        min_cosim_val = min(cosim_vals)
        min_cosim_idx = cosim_vals.index(min_cosim_val)
        min_cosims.append(min_cosim_val)
        min_question_ids.append(min_cosim_idx + 1)

    df['closest_answer_id'] = min_question_ids
    df['closest_cosim'] = min_cosims
    return df 

def get_right_wrong(df: pd.DataFrame) -> pd.DataFrame: 
    df['archq_closest'] = (df['answer_id'] == df['closest_answer_id']).astype('int64')
    return df

pairwise_eval_df = (
    eval_df[[
        'question_id', 
        'question_text', 
        'question_embedding', 
        'group',
        'answer_id',
        'answer_text',
        'archetype_question',
    ]].copy(deep=True)
    .pipe(calc_pairwise_distances)
    .pipe(get_shortest_dist)
    .pipe(get_right_wrong)
    .pipe(pd.DataFrame.rename, columns={
        "answer_text": "archetype_answer", 
        "answer_id": "archetype_answer_id"
    })
)

final_evaluation_df = (
    pairwise_eval_df[[
        'question_id', 
        'question_text', 
        'archetype_question',
        'archetype_answer',
        'group', 
        'archetype_answer_id',
        'archq1_cosim',
        'archq2_cosim',
        'archq3_cosim',
        'archq4_cosim', 
        'closest_answer_id',
        'closest_cosim',
        'archq_closest'
    ]]
    # .pipe(lambda df: df[df.archq_closest == 0])
)

final_evaluation_df

,question_id,question_text,archetype_question,archetype_answer,group,archetype_answer_id,archq1_cosim,archq2_cosim,archq3_cosim,archq4_cosim,closest_answer_id,closest_cosim,archq_closest
0,50,Is het toegestaan om in een academisch ziekenh...,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",location,1,0.065959,0.116500,0.112070,0.148158,1,0.065959,1
1,150,"Baby oké, nu naar huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.146958,0.140031,0.147055,0.209457,2,0.140031,0
2,149,"Baby klaar, weg nu?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.171928,0.160489,0.160960,0.206592,2,0.160489,0
3,148,"Baby er, snel huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.138975,0.137391,0.113837,0.191029,3,0.113837,0
4,147,"Baby geboren, ik naar huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.095512,0.124378,0.079105,0.157814,3,0.079105,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,494,Is er iets dat ouders moeten doen of weten met...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,location,4,0.174044,0.134112,0.135245,0.111524,4,0.111524,1
595,495,Wat zijn de belangrijkste overwegingen bij het...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,location,4,0.176198,0.147297,0.140297,0.119217,4,0.119217,1
596,496,Kunnen dezelfde zorgmaatregelen worden genomen...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,location,4,0.163606,0.129979,0.139844,0.124013,4,0.124013,1
597,497,Wat moeten ouders doen als ze zich zorgen make...,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,location,4,0.183635,0.147891,0.143334,0.127660,4,0.127660,1


# Evaluation

The following pandas data frame summarizes the cosine similarity for a particular question_text to other archetype questions (`archq{i}`_cosim columns). If the `question_text` is closest to its actual archetype question, `closest_answer_id` matches `archetype_answer_id` and `archq_closest` is 1. Otherwise `archq_closest` is 0. 

In [125]:
final_evaluation_df.head(5)

,question_id,question_text,archetype_question,archetype_answer,group,archetype_answer_id,archq1_cosim,archq2_cosim,archq3_cosim,archq4_cosim,closest_answer_id,closest_cosim,archq_closest
0,50,Is het toegestaan om in een academisch ziekenh...,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",location,1,0.065959,0.116500,0.112070,0.148158,1,0.065959,1
1,150,"Baby oké, nu naar huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.146958,0.140031,0.147055,0.209457,2,0.140031,0
2,149,"Baby klaar, weg nu?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.171928,0.160489,0.160960,0.206592,2,0.160489,0
3,148,"Baby er, snel huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.138975,0.137391,0.113837,0.191029,3,0.113837,0
4,147,"Baby geboren, ik naar huis?",Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",low_literacy,1,0.095512,0.124378,0.079105,0.157814,3,0.079105,0


## Cosine Distributions 

Distribution of Cosine Similarity Scores: Histograms or box plots for each archq{i}_cosim column to see the distribution of cosine similarity scores. This can help identify if certain archetypes are consistently scoring higher or lower.

In [135]:
import plotly.express as px


for i in answer_df['answer_id']: 
    # Assuming your DataFrame is named df
    fig = px.histogram(
        final_evaluation_df, 
        x=f"archq{i}_cosim", 
        nbins=30, 
        title=f"Distribution of Cosine Similarity for Archetype {i}"
    )
    fig.show()


## Comparison
Comparison of Closest Cosine Similarity to Others: A scatter plot comparing closest_cosim with other archq{i}_cosim values. This will help visualize if the closest matches are significantly higher than the other archetype matches.

In [138]:
for i in answer_df['answer_id']: 
    fig = px.scatter(
        final_evaluation_df, 
        x="closest_cosim", 
        y=f"archq{i}_cosim", 
        title=f"Comparison of Closest Cosine Similarity to Archetype {i}"
    )
    fig.show()



## Correct vs Incorrect

Correct vs Incorrect Matches: A bar chart showing the count of correct vs incorrect matches (based on archq_closest). This provides a quick overview of the matching accuracy.

In [129]:
correct_counts = final_evaluation_df['archq_closest'].value_counts()
fig = px.bar(
    correct_counts, 
    x=correct_counts.index, 
    y=correct_counts.values, 
    title="Correct vs Incorrect Archetype Matches",
    labels={'x': 'Match Correct', 'y': 'Count'}
)
fig.show()


## Group Analysis
Group-Wise Analysis: If the group column categorizes questions in a meaningful way, examining distributions or counts split by group can be insightful.

In [130]:
fig = px.histogram(
    final_evaluation_df, 
    x="group", 
    color="archq_closest", 
    barmode='group',
    title="Group-wise Correct vs Incorrect Matches"
)
fig.show()
